In [1]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.compat.v1.keras import layers, models  # Use keras within tensorflow.compat.v1
import tensorflow.compat.v1 as tf

In [3]:
# Load your CSV file
csv_file_path = "output.csv"
df = pd.read_csv(csv_file_path,sep='\t', header=None)

In [4]:
df.head(5)

,0
0,"0;[116.19, 116.19, 116.19, 5780.0, 381.32, -10..."
1,"0;[165.59, 165.59, 165.59, -3380.0, 93.41, -11..."
2,"0;[163.16, 163.16, 163.16, 1780.0, -308.79, -1..."
3,"0;[365.18, 365.18, 365.18, -1720.0, 186.81, -1..."
4,"0;[26.72, 26.72, 26.72, -1560.0, 270.33, 130.2..."


In [5]:
# Extract labels from the first column
labels = df[0].str.extract(r'(\d+)', expand=False).astype(float)

# Extract features from the second column
features = df[0].str.extractall(r'(\d+\.\d+|-?\d+\.\d+|-?\d+)').unstack().astype(float)

# Concatenate labels and features into a new DataFrame
processed_df = pd.concat([labels, features], axis=1)

In [7]:
# Display the processed DataFrame
print("Processed DataFrame:")
print(processed_df.head(5))

Processed DataFrame:
     0  (0, 0)  (0, 1)  (0, 2)  (0, 3)  (0, 4)  (0, 5)  (0, 6)  (0, 7)  \
0  0.0     0.0  116.19  116.19  116.19  5780.0  381.32 -102.55  114.17   
1  0.0     0.0  165.59  165.59  165.59 -3380.0   93.41 -117.02  117.98   
2  0.0     0.0  163.16  163.16  163.16  1780.0 -308.79  -17.45  119.62   
3  0.0     0.0  365.18  365.18  365.18 -1720.0  186.81 -137.02  193.46   
4  0.0     0.0   26.72   26.72   26.72 -1560.0  270.33  130.21   37.60   

   (0, 8)  ...  (0, 3639)  (0, 3640)  (0, 3641)  (0, 3642)  (0, 3643)  \
0   24.93  ...     -69.76      -9.18     -402.4     140.94     5540.0   
1  217.89  ...      -3.45      57.31       47.2     378.74     -740.0   
2  272.43  ...      96.82      54.98      234.4     317.32     -380.0   
3   90.91  ...     -10.08      61.84       27.2      96.06    -2440.0   
4  121.41  ...     122.55      93.92      -43.2     307.09    -6120.0   

   (0, 3644)  (0, 3645)  (0, 3646)  (0, 3647)  (0, 3648)  
0      71.22      -0.15     152.14  

In [8]:
# Extract numeric labels using regular expression
processed_df[0] = processed_df[0].apply(lambda x: re.search(r'-?\d+\.\d+', str(x)).group()).astype(float)

In [9]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
print(y_train.dtype)
print(y_test.dtype)


object
object


In [12]:
y_train = pd.to_numeric(y_train, errors='coerce')
y_test = pd.to_numeric(y_test, errors='coerce')


In [13]:
print(y_train.dtype)
print(y_test.dtype)

float64
float64


In [14]:
# Build a simple neural network model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
])

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [16]:
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/50



: 

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')